In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
from scipy import sparse
import loompy as lp
from loomxpy._loomx import LoomX

## Data

In [2]:
adata = sc.read_h5ad("../data/test.h5ad")

## TL;TR

In [3]:
lx = LoomX()
lx.modes.rna = (adata.raw.X, adata.raw.var.index, adata.obs.index)
lx.modes.rna.o.annotations.batch = pd.DataFrame({"batch": adata.obs.batch}, index=adata.obs.index)
lx.modes.rna.o.embeddings.pca = pd.DataFrame(adata.obsm["X_pca"], index=adata.obs.index)
lx.modes.rna.o.clusterings.leiden_1_0 = pd.DataFrame(adata.obs["leiden_res1.0"], index=adata.obs.index)
lx.modes.rna.o.get_attribute(key="leiden_1_0").name = "Leiden res 1.0"
lx.modes.rna.export(filename="../data/test.loom", output_format="scope_v1")

INFO: adding new rna mode
INFO: LoomX successfully exported to SCope-compatible loom file.


## Init

In [4]:
lx = LoomX()

In [5]:
lx.modes

Modalities: none

## Add Mode

In [6]:
lx.modes.rna = (adata.raw.X, adata.raw.var.index, adata.obs.index)

INFO: adding new rna mode


## Add Feature Attributes

### Add annotations

In [7]:
hvg_df = pd.DataFrame(index=adata.raw.var.index).merge(adata.var.highly_variable, left_index=True, right_index=True, how="left").fillna(value=False)
hvg_df.head(n=1)

,highly_variable
index,
128up,False


In [8]:
# Recommended way of adding annotations
lx.modes.rna.f.annotations.hvg = hvg_df

In [9]:
lx.modes.rna.f.annotations._keys

['hvg']

In [10]:
lx.modes.rna.f.annotations.hvg.head(n=1)

,highly_variable
index,
128up,False


In [11]:
lx.modes.rna.f.annotations["hvg"].head(n=1)

,highly_variable
index,
128up,False


In [12]:
lx.modes.rna.f["hvg"].head(n=1)

,highly_variable
index,
128up,False


In [13]:
lx.modes.rna.f.hvg.head(n=1)

,highly_variable
index,
128up,False


### Add Metrics

In [14]:
dispersions_df = pd.DataFrame(index=adata.raw.var.index).merge(adata.var.dispersions, left_index=True, right_index=True, how="left")
dispersions_df.head(n=1)

,dispersions
index,
128up,NaN


In [15]:
# Recommended way of adding metrics
lx.modes.rna.f.metrics.disp = dispersions_df

In [16]:
# Recommended way of adding metrics
lx.modes.rna.f.metrics["disp"] = dispersions_df

In [17]:
lx.modes.rna.f.metrics.disp.head(n=1)

,dispersions
index,
128up,NaN


In [18]:
lx.modes.rna.f.metrics["disp"].head(n=1)

,dispersions
index,
128up,NaN


In [19]:
lx.modes.rna.f.disp.head(n=1)

,dispersions
index,
128up,NaN


In [20]:
lx.modes.rna.f["disp"].head(n=1)

,dispersions
index,
128up,NaN


## Add Observation Features

### Add Annotations

In [21]:
age_df = pd.DataFrame({"age": adata.obs.age}, index=adata.obs.index)
age_df.head(n=1)

,age
AAACCCACATGACGGA,5


In [22]:
# Recommended way of annotations
lx.modes.rna.o.annotations.batch = pd.DataFrame({"batch": adata.obs.batch}, index=adata.obs.index)

In [23]:
# Trying to add annotation with dtype int
lx.modes.rna.o.annotations.age = age_df

Exception: Expects value to be categorical or bool but its dtype is int64. You can force the conversion to categorical by using <loomx-instance>.modes.<mode>.annotations.add(*, force=True).

In [24]:
# Recommended way of adding annotations in case dtype is number
lx.modes.rna.o.annotations.add(key="age", value=age_df, force=True)

/ddn1/vol1/staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/GitHub/LoomXpy/loomxpy/_mode.py:614: UserWarning: Converting age annotation to categorical type...
  warnings.warn(f"Converting {name} annotation to categorical type...")


In [25]:
lx.modes.rna.o.annotations._keys

['batch', 'age']

In [26]:
lx.modes.rna.o.annotations.age.head(n=1)

,age
AAACCCACATGACGGA,5


In [27]:
lx.modes.rna.o.annotations["age"].head(n=1)

,age
AAACCCACATGACGGA,5


### Add metrics 

In [28]:
n_counts_df = pd.DataFrame({"n_counts": adata.obs.n_counts}, index=adata.obs.index)
n_counts_df.head(n=1)

,n_counts
AAACCCACATGACGGA,852.0


In [29]:
lx.modes.rna.o.metrics.n_counts = n_counts_df

In [30]:
lx.modes.rna.o.n_counts.head(n=1)

,n_counts
AAACCCACATGACGGA,852.0


In [31]:
lx.modes.rna.o.metrics["n_counts"].head(n=1)

,n_counts
AAACCCACATGACGGA,852.0


### Add embeddings

In [32]:
x_pca_df = pd.DataFrame(adata.obsm["X_pca"], index=adata.obs.index)

In [33]:
x_pca_df.head(n=1)

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
AAACCCACATGACGGA,0.171696,-0.388365,-3.832047,8.717521,-5.364988,-1.2397,-1.463515,1.278344,0.483982,1.349265,...,-0.718103,-0.168473,2.005419,0.64762,-2.659557,0.481644,-2.572921,1.974173,-1.734001,-1.356891


In [34]:
lx.modes.rna.o.embeddings.pca = x_pca_df

In [35]:
lx.modes.rna.o.embeddings.add(key="pca", value=x_pca_df, name="PCA", description="Principal component analysis [Pedregosa11].")

In [36]:
lx.modes.rna.o.get_attribute(key="pca")



key: pca
mode: ModeType.RNA
type: AttributeType.EMBEDDING
name: PCA
description: Principal component analysis [Pedregosa11].
        
projection method: PCA
        

In [37]:
lx.modes.rna.o.embeddings.pca.head(n=1)

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
AAACCCACATGACGGA,0.171696,-0.388365,-3.832047,8.717521,-5.364988,-1.2397,-1.463515,1.278344,0.483982,1.349265,...,-0.718103,-0.168473,2.005419,0.64762,-2.659557,0.481644,-2.572921,1.974173,-1.734001,-1.356891


### Add clusterings

In [38]:
l_df = adata.obs["leiden_res1.0"]

In [39]:
lx.modes.rna.o.clusterings.leiden_1_0 = pd.DataFrame(l_df, index=adata.obs.index)

In [40]:
lx.modes.rna.o.clusterings.leiden_1_0.head(n=1)

,leiden_res1.0
AAACCCACATGACGGA,10


In [41]:
cl = lx.modes.rna.o.clusterings.get_attribute("leiden_1_0")
cl.name = "Leiden res 1.0"

In [42]:
cl.cluster_0.name = "This a cluster X (previously 0)"

In [43]:
cl.cluster_0.name

'This a cluster X (previously 0)'

## Set Active Mode

In [44]:
lx.modes

Modalities: rna

In [45]:
lx.active = "rna"

In [46]:
lx.f.hvg.head(n=1)

,highly_variable
index,
128up,False


## Export

### SCope

In [47]:
lx.modes.rna.export(filename="../data/test.loom", output_format="scope_v1")

INFO: LoomX successfully exported to SCope-compatible loom file.


## Env

In [48]:
pip freeze

aiohttp==3.7.4
alabaster==0.7.12
anndata==0.7.4
appdirs==1.4.4
arboreto==0.1.6
argon2-cffi==20.1.0
arrow==1.0.2
astroid==2.5.1
async-generator==1.10
async-timeout==3.0.1
attrs==20.2.0
Babel==2.9.0
backcall==0.2.0
binaryornot==0.4.4
black==20.8b1
bleach==3.2.1
boltons==20.2.1
certifi==2020.6.20
cffi==1.14.3
chardet==3.0.4
click==7.1.2
cloudpickle==1.6.0
cookiecutter==1.7.2
cycler==0.10.0
cytoolz==0.11.0
dask==2021.2.0
decorator==4.4.2
defusedxml==0.6.0
dill==0.3.3
distributed==2021.2.0
docutils==0.16
entrypoints==0.3
frozendict==1.2
fsspec==0.8.7
get-version==2.1
h5py==2.10.0
HeapDict==1.0.1
idna==2.10
imagesize==1.2.0
importlib-metadata==2.0.0
interlap==0.2.7
ipykernel==5.3.4
ipython==7.18.1
ipython-autotime==0.1
ipython-genutils==0.2.0
isort==5.7.0
jedi==0.17.2
Jinja2==2.11.2
jinja2-time==0.2.0
joblib==0.16.0
json5==0.9.5
jsonschema==3.2.0
jupyter-client==6.1.7
jupyter-core==4.6.3
jupyterlab==2.2.8
jupyterlab-pygments==0.1.2
jupyterlab-server==1.2.0
kiwisolver==1.2.0
lazy-object-proxy